In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [2]:
from ipynb.fs.full.Yahoo_Finance_functions import modify_ticker, get_many_tickers, get_stock_returns,\
removeall_replace

In [3]:
my_file = pd.read_csv('stockchase_datetime.csv')
my_file = my_file.iloc[:, 2:]

In [4]:
my_file.Ticker=my_file.Ticker.apply(lambda x: modify_ticker(x))

In [5]:
my_file.Date = my_file.Date.apply(lambda x: datetime.strptime(x, "%Y-%m-%d").date())

In [6]:
# my_file.Date[0] + relativedelta(months=2)

In [7]:
# ticker_list = my_file[my_file.Date==date(year=2019, month=10, day=1)].Ticker.tolist()
# ticker_list

In [8]:
# start_date = date(year=2019, month=7, day=2)
# end_date = start_date + relativedelta(months=2)
# stock_data = get_many_tickers(ticker_list, start_date, end_date)

In [9]:
# start_date = stock_data['Close'].index.date[0]
# end_date = stock_data['Close'].index.date[-1]

In [10]:
unique_dates = my_file.Date.unique()
unique_dates = unique_dates[::-1]
unique_dates

array([datetime.date(2000, 10, 2), datetime.date(2000, 10, 3),
       datetime.date(2000, 10, 4), ..., datetime.date(2019, 9, 27),
       datetime.date(2019, 9, 30), datetime.date(2019, 10, 1)],
      dtype=object)

In [11]:
def get_ticker_list(date, my_file=my_file):
    ticker_list = my_file[my_file.Date==date].Ticker.tolist()
    return ticker_list

In [12]:
portfolio_series = pd.Series(unique_dates).apply(get_ticker_list)

In [13]:
portfolio_series = portfolio_series.apply(lambda x: removeall_replace(x))

In [14]:
BAD_TICKERS = ['NT.TO', 'AUR.TO', 'ITC.TO', 'TWF-UN.TO', 'TRI.TO', 'GND.TO', 'MSCC', 'WRX.TO', 'IW.TO', 'SNC.TO', 
              'SSB.TO', 'HW.TO', 'CX.TO', 'MAE.TO', 'IDB.TO', 'LU', 'MG.TO', 'ICOS', 'FSH.TO', 'EMC', 'ABI', 'DTC.TO',
              'PCA.TO', 'ZL.TO', 'BPO.TO', 'STE.TO', 'HSP', 'LGY-UN.TO', 'VSN.TO', 'CSN.TO', 'SJR-B.TO', 'GTW', 'CYTC',
              'AFFX', 'KFS.TO', 'TUN.TO', 'SHC.TO', 'AAC-B.TO', 'LTV.TO', 'SMN-UN.TO', 'Q', 'AOM.TO', 'OPMR', 'HKY.TO',
              'MIFI', 'VAS.TO', 'IQW', 'BAM-LV-A.TO', 'NS.TO', 'FDC', 'HML.TO', 'DELL']

In [15]:
def fliter_bad_stocks(portfolio_series, unique_dates, BAD_TICKERS=BAD_TICKERS):
    
    for bad_ticker in BAD_TICKERS:
            portfolio_series = portfolio_series.apply(lambda x: removeall_replace(x, my_element=bad_ticker))
    
    empty_rows_bool = portfolio_series.apply(lambda x: len(x)>0)
    portfolio_series = portfolio_series[empty_rows_bool].reset_index(drop=True)
    unique_dates = unique_dates[empty_rows_bool]
    
    return portfolio_series, unique_dates

In [16]:
portfolio_series, unique_dates = fliter_bad_stocks(portfolio_series, unique_dates)

In [17]:
def rolling_portfolio(portfolio_series, unique_dates, portfolio_size=60):
    n_total_days = len(portfolio_series)
    portfolio_returns=np.ones(portfolio_size)
    
    for n_days in tqdm(range(0, n_total_days - portfolio_size)):
        last_portfolio_day = min(n_days + portfolio_size, n_total_days)
        
        current_portfolio = portfolio_series[n_days: last_portfolio_day]
        current_dates = unique_dates[n_days: last_portfolio_day]
#         print(current_portfolio)
        oldest_portfolio = current_portfolio[n_days]
        oldest_portfolio_start_date = current_dates[n_days]
        oldest_portfolio_end_date = current_dates[-1]
        
        new_stock_data = get_many_tickers(oldest_portfolio, oldest_portfolio_start_date, oldest_portfolio_end_date)
        
        new_returns_df = get_stock_returns(new_stock_data, oldest_portfolio_start_date, oldest_portfolio_end_date)
        new_return = new_returns_df.mean(axis=1)
        portfolio_returns[n_days%portfolio_size]=portfolio_returns[n_days%portfolio_size]*(1 + new_return)
    return portfolio_returns

In [18]:
# small_port_ret = rolling_portfolio(portfolio_series[0:200], unique_dates[0:200])

check the returns 60 trading days later and take the sum of all the individual portfolios

In [ ]:
bad_portfolios=[]
portfolio_size=60
n_total_days = len(portfolio_series)

for n_days in tqdm(range(0, n_total_days)):
    try:
        oldest_portfolio = portfolio_series[n_days]
        oldest_portfolio_start_date = unique_dates[n_days]
        oldest_portfolio_end_date = unique_dates[n_days+20]

        new_stock_data = get_many_tickers(oldest_portfolio, oldest_portfolio_start_date, oldest_portfolio_end_date)
    except:
        bad_portfolios.append([oldest_portfolio, n_days])

  0%|          | 0/4681 [00:00<?, ?it/s]

[*********************100%***********************]  1 of 1 downloaded

  0%|          | 1/4681 [00:01<1:18:59,  1.01s/it]


[*********************100%***********************]  1 of 1 downloaded

  0%|          | 2/4681 [00:01<1:10:14,  1.11it/s]

...

[*********************100%***********************]  4 of 5 downloaded

Exception in thread Thread-4943:
Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init__.py", line 363, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init__.py", line 208, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init

Exception in thread Thread-4948:
Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/urllib3/contrib/pyopenssl.py", line 453, in wrap_socket
    cnx.do_handshake()
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1915, in do_handshake
    self._raise_ssl_error(self._ssl, result)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/OpenSSL/SSL.py", line 1639, in _raise_ssl_error
    raise SysCallError(errno, errorcode.get(errno))
OpenSSL.SSL.SysCallError: (54, 'ECONNRESET')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 343, in _make_request
    self._validate_conn(conn)
  File "/Users/Yohan/Des

[*******************   40%                       ]  2 of 5 downloaded

Exception in thread Thread-4951:
Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init__.py", line 363, in history
    quotes = self._parse_quotes(data["chart"]["result"][0])
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init__.py", line 208, in _parse_quotes
    timestamps = data["timestamp"]
KeyError: 'timestamp'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/multitasking/__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "/Users/Yohan/Desktop/anaconda3/lib/python3.7/site-packages/yfinance/__init